In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import rho_plus as rp

is_dark = True
theme, cs = rp.mpl_setup(is_dark=is_dark)

In [25]:
origin = np.array([0, 0, 1/6])
hull_pts = np.array([( 1/24,-1/4, 1/12),
(-5/24, 0,-1/6),
( 0, 1/4, 1/12),
( 0,-1/4, 1/12),
(-1/24,-1/8,-5/24),
( 1/8, 5/24, 1/24),
(-1/8,-1/24,-5/24),
(-1/24, 1/4, 1/12),
( 1/4,-1/24, 1/12),
( 1/24, 1/4, 1/12),
( 5/24, 1/8, 1/24),
( 0,-1/4,-1/24),
( 0,-5/24,-1/6),
(-1/8, 0, 1/12),
( 1/4, 0, 1/12),
(-1/24,-1/4, 1/12),
( 1/4, 1/24, 1/12)])

In [28]:
import gemmi
from cctbx.sgtbx.direct_space_asu.reference_table import get_asu
import pyvista as pv
from scipy.spatial import ConvexHull, Delaunay

pv.set_jupyter_backend('trame')


if is_dark:
    pv.set_plot_theme('dark')
else:
    pv.set_plot_theme('default')

p: pv.plotting.Plotter = pv.Plotter()

sg = gemmi.SpaceGroup(220)
print(sg)
asu = get_asu(sg.number)

def plot_plane(cut, **kwargs):
    pl = cut.as_float_cut_plane()
    ori = pl.get_point_in_plane()
    p.add_mesh(pv.Plane(ori, pl.n), **kwargs)

# for cut, color in zip(asu.cuts, cs):
#     plot_plane(cut, color=color, opacity=0.5)

n_neighbors = 1

for op in sg.operations():
    im = op.apply_to_xyz(origin)
    im_mod = (im - origin + 0.51) % 1 + origin - 0.51
    im_mod2 = (im - origin + 0.48) % 1 + origin - 0.48
    p.add_mesh(pv.Sphere(0.05, im_mod))
    if np.abs(im_mod2 - im_mod).max() >= 0.5:
        p.add_mesh(pv.Sphere(0.05, im_mod), color=cs[2])

# hull = ConvexHull(asu.shape_vertices())
hull = ConvexHull(hull_pts + origin)
grid = hull.simplices

lat = pv.PolyData.from_regular_faces(hull.points, hull.simplices)
p.add_mesh(lat, opacity=0.6)

p.show()

<gemmi.SpaceGroup("I -4 3 d")>


Widget(value="<iframe src='http://localhost:38677/index.html?ui=P_0x76bfae9ae710_19&reconnect=auto' style='wid…

In [154]:
import Geometry3D as g3

def to_geo(cut):
    pl = cut.as_float_cut_plane()
    ori = pl.get_point_in_plane()
    return g3.Plane(g3.Point(ori), g3.Vector(pl.n))

pt = g3.Point(np.mean(hull.points, axis=0))

cut = asu.cuts[0]
pl = to_geo(cut)
ref_line = g3.Line(pt, pl.n)
proj = g3.intersection(ref_line, pl)
mirror = g3.Point(pt.pv() + 2 * (pt.pv() - proj.pv()))

pvec = np.array(pt.pv()._v)
mvec = np.array(mirror.pv()._v)


In [156]:
for op in sg.operations().sym_ops:
    op: gemmi.Op = op
    out = np.array(op.apply_to_xyz(pvec))
    print(op, out - mvec)
    if np.abs((out - mvec) % 1).sum() <= 1e-1:
        print(op)
        break

mvec - op.apply_to_xyz(pvec)

<gemmi.Op("x,y,z")> [ 0.16666667  0.         -0.16666667]
<gemmi.Op("-y+3/4,x+1/4,z+3/4")> [0.33333333 0.125      0.58333333]
<gemmi.Op("-x+1/2,-y,z+1/2")> [ 0.20833333 -0.70833333  0.33333333]
<gemmi.Op("y+3/4,-x+3/4,z+1/4")> [1.04166667 0.16666667 0.08333333]
<gemmi.Op("x+1/2,-y+1/2,-z")> [ 0.66666667 -0.20833333  0.04166667]
<gemmi.Op("-y+1/4,-x+1/4,-z+1/4")> [-0.16666667 -0.33333333  0.29166667]
<gemmi.Op("-x,y+1/2,-z+1/2")> [-0.29166667  0.5         0.54166667]
<gemmi.Op("y+1/4,x+3/4,-z+3/4")> [0.54166667 0.625      0.79166667]
<gemmi.Op("z,x,y")> [-0.16666667 -0.125       0.29166667]
<gemmi.Op("z+3/4,-y+3/4,x+1/4")> [0.58333333 0.04166667 0.41666667]
<gemmi.Op("z+1/2,-x+1/2,-y")> [ 0.33333333 -0.08333333 -0.41666667]
<gemmi.Op("z+1/4,y+3/4,-x+3/4")> [0.08333333 0.75       0.45833333]
<gemmi.Op("-z,x+1/2,-y+1/2")> [0.04166667 0.375      0.08333333]
<gemmi.Op("-z+1/4,-y+1/4,-x+1/4")> [ 0.29166667 -0.45833333 -0.04166667]
<gemmi.Op("-z+1/2,-x,y+1/2")> [ 0.54166667 -0.58333333  0.791

array([-0.08333333,  0.25      , -0.66666667])

In [161]:
asu.show_comprehensive_summary()

Hall symbol: P 4acd 2ab 3
Number of cuts: 6
    & zx2
    & -yz0(-zx2)
    & ~yz2(tx0)
    & -tx0(x8)
    & ty0(y8)
    & tz2(-x1*3/8)
